In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, Random, Distributions, DataFrames


In [2]:
function generateDistribution(n, p)
    m = floor(Int, n*p)
    a = ones(Int, m)
    b = zeros(Int, n-m)
    res = [a;b]
    return shuffle(res)
end

generateDistribution (generic function with 1 method)

In [3]:
function setDepth(n)
    if n < 12
        return [3, 4, 5]
    else
        return [4, 5, 6, 7]
    end
end

setDepth (generic function with 1 method)

In [ ]:
print("hello")

In [4]:
seed = 15095
p = 0.05
results = zeros(80, 4)
num_result = 1
for n=[4:2:32;]
    test = Matrix(I, n, n);
    m = floor(Int,4*n*((0.5-p)^(-2)))
    threshold = ceil(Int, m*p + sqrt(n*m))
    for ratio in [0, 0.125, 0.25, 0.375, 0.5]
        s = generateDistribution(n, ratio)
        count = 0
        for i=1:25
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Train RF
            lnr = IAI.XGBoostClassifier(random_seed=seed) 
            grid = IAI.GridSearch(lnr, max_depth=setDepth(n), num_rounds=[20, 50, 100])
            time = @elapsed IAI.fit!(grid, A, b)
            best_lnr = IAI.get_learner(grid)
            auc = round(IAI.score(best_lnr, A, b, positive_label=1, criterion=:auc), digits=2)
            s_pred = IAI.predict(best_lnr, test);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end

            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, i=$i, auc=$auc, error=$error")        
        end   
        result = [n, m, threshold, ratio, count, time]
        results[num_result, :] = result
        num_result += 1
    end
end




LoadError: UndefVarError: False not defined

In [ ]:
df = DataFrame(results, :auto)
CSV.write("results_xgb.csv",df)